# Spokane Solar Suitability
### Areeba Usman & Rose Martin
### 
---

## Section 0: Load in data and import packages

In [ ]:
!pip install -q censusdata

In [ ]:
import os
import shutil
import zipfile
import requests
import censusdata
import rasterstats
import numpy as np
import pandas as pd
import xarray as xr
import seaborn as sns
import rasterio as rio
import geopandas as gpd
import rioxarray as rxr
from pathlib import Path
import contextily as ctx
from scipy.ndimage import label
import matplotlib.pyplot as plt
from scipy.spatial import cKDTree
import xyzservices.providers as xyz
from rasterio.features import rasterize
from rasterio import plot, mask, features
from rasterio.transform import from_origin
from matplotlib_scalebar.scalebar import ScaleBar
from shapely.geometry import Point, Polygon, shape, box
from rasterio.warp import calculate_default_transform, reproject, Resampling

---
## Section 1: Loading Zone
### Census Data

In [ ]:
#Functions taken from lab 5
def get_census_data(tables, state, county, year=2019):
    '''Download census data for a given state and county fips code.'''

    # Download the data
    data = censusdata.download('acs5', year,  # Use 2019 ACS 5-year estimates
                               censusdata.censusgeo([('state', state), ('county', county), ('tract', '*')]),
                               list(tables.keys()))

    # Rename the column
    data.rename(columns=tables, inplace=True)

    # Extract information from the first column
    data['Name'] = data.index.to_series().apply(lambda x: x.name)
    data['SummaryLevel'] = data.index.to_series().apply(lambda x: x.sumlevel())
    data['State'] = data.index.to_series().apply(lambda x: x.geo[0][1])
    data['County'] = data.index.to_series().apply(lambda x: x.geo[1][1])
    data['Tract'] = data.index.to_series().apply(lambda x: x.geo[2][1])
    data.reset_index(drop=True, inplace=True)
    data = data[['Tract','Name']+list(tables.values())].set_index('Tract')
    
    return data

def get_census_tract_geom(state_fips, county_fips):
    '''Download census tract geometries for a given state and county fips code, storing in /tmp and cleaning up after.'''

    temp_dir = "/tmp/census_tracts"
    zip_path = os.path.join(temp_dir, f'tl_2019_{state_fips}_tract.zip')

    # Ensure temp directory exists
    os.makedirs(temp_dir, exist_ok=True)

    # Download the file
    url = f'https://www2.census.gov/geo/tiger/TIGER2019/TRACT/tl_2019_{state_fips}_tract.zip'
    response = requests.get(url, stream=True)
    if response.status_code != 200:
        raise Exception(f"Failed to download file: {url}")

    # Save ZIP file to temp directory
    with open(zip_path, "wb") as file:
        file.write(response.content)

    # Extract the ZIP file
    with zipfile.ZipFile(zip_path, "r") as zip_ref:
        zip_ref.extractall(temp_dir)

    # Find the shapefile in extracted contents
    for file in os.listdir(temp_dir):
        if file.endswith(".shp"):
            shapefile_path = os.path.join(temp_dir, file)
            break

    # Read the shapefile into a GeoDataFrame
    tracts = gpd.read_file(shapefile_path)

    # Filter by county and set index
    tracts = tracts[tracts['COUNTYFP'] == county_fips]
    tracts = tracts.rename(columns={'TRACTCE': 'Tract'}).set_index('Tract')

    # Cleanup: Remove extracted files and ZIP file
    shutil.rmtree(temp_dir)

    return tracts[['geometry']]



In [ ]:
tables = {
'B19013_001E': 'MedianIncome',
'B01003_001E': 'TotalPopulation',
'B25064_001E': 'MedianGrossRent',
'B25077_001E': 'MedianHomeValue',
'B25035_001E': 'MedianYearStructureBuilt',
'B25001_001E': 'TotalHousingUnits',
'B25004_001E': 'TotalVacantHousingUnits',
'B25003_002E': 'OccupiedHousingUnitsOwnerOccupied',
'B25003_003E': 'OccupiedHousingUnitsRenterOccupied',
}

In [ ]:
## FIPS Code for Washington
state_fips = '53' 

## FIPS code for Spokane County
county_fips = '063'  

In [ ]:
census_df = get_census_data(tables, state_fips, county_fips)
tract_geom_gdf = get_census_tract_geom(state_fips, county_fips)
tract_geom_gdf_fo_proj = tract_geom_gdf

In [ ]:
census_Spokane_gdf = gpd.GeoDataFrame(census_df.join(tract_geom_gdf))
census_Spokane_gdf =  census_Spokane_gdf.to_crs('epsg:32610')
#census_df.head()

#### AEA Projection around Spokane

In [ ]:
hull = tract_geom_gdf_fo_proj.geometry.unary_union.convex_hull
cent = hull.centroid

cent_lon =cent.x
cent_lat = cent.y
min_lat = tract_geom_gdf_fo_proj.bounds["miny"].min().item()
max_lat = tract_geom_gdf_fo_proj.bounds["maxy"].max().item()

In [ ]:
proj_str_aea = f'+proj=aea +lat_1={min_lat:0.2f} +lat_2={max_lat:0.2f} +lat_0={cent_lat:0.2f} +lon_0={cent_lon:0.2f}'
print(proj_str_aea)

### States

In [ ]:
states_url = 'http://eric.clst.org/assets/wiki/uploads/Stuff/gz_2010_us_040_00_500k.json'
states_gdf = gpd.read_file(states_url)
states_gdf.head()

In [ ]:
## 32048 is the EPSG recommended for Spokane County
states_proj_gdf = states_gdf.to_crs(4326)

WA_idx = states_proj_gdf["NAME"].isin(["Washington"])
wa_state_gdf = states_proj_gdf[WA_idx]

### Flood Plains

In [ ]:
floodplains_fn = './Data/FEMA_Flood_Zone.geojson'
floodplains_gdf = gpd.read_file(floodplains_fn)

In [ ]:
floodplains_gdf = floodplains_gdf.set_crs(4326)
floodplains_gdf.head()

#### Binary Flood Plain Raster

In [ ]:
# Reproject to projected CRS
floodplains_gdf = floodplains_gdf.to_crs(proj_str_aea)
tract_geom_gdf = tract_geom_gdf.to_crs(proj_str_aea)

# Get bounds of the area of interest
xmin, ymin, xmax, ymax = tract_geom_gdf.total_bounds

# Define the resolution of the raster (in meters)
resolution = 100  # Change as needed
width = int((xmax - xmin) / resolution)
height = int((ymax - ymin) / resolution)

# Define transform for rasterization
transform = from_origin(xmin, ymax, resolution, resolution)

# Create binary raster mask of floodplain areas (value = 0)
shapes_floodplain = [(geom, 0) for geom in floodplains_gdf.geometry]
raster = rasterize(shapes_floodplain, out_shape=(height, width), transform=transform, fill=1, dtype=np.float32)

# Create a 500-meter buffer around floodplain areas
floodplains_gdf["geometry"] = floodplains_gdf.geometry.buffer(500)

# Rasterize the buffered floodplain areas (value = 0.5), but only overwrite areas that are still 1
shapes_buffer = [(geom, 0.5) for geom in floodplains_gdf.geometry]
buffer_raster = rasterize(shapes_buffer, out_shape=(height, width), transform=transform, fill=1, dtype=np.float32)

# Merge rasters: Keep original 0s, overwrite 1s with buffer values (0.5)
raster[(raster == 1) & (buffer_raster == 0.5)] = 0.5

# Save raster to file
with rio.open(
    "./Data/floodplain_with_buffer.tif", "w", driver="GTiff", height=height, width=width,
    count=1, dtype=np.float32, crs=proj_str_aea, transform=transform
) as dst:
    dst.write(raster, 1)


In [ ]:
floodplain_buffer_fn = "./Data/floodplain_with_buffer.tif"
floodplain_buffer_data = rxr.open_rasterio(floodplain_buffer_fn).squeeze()

### Slope Data

In [ ]:
dst_crs = 'EPSG:32610'

In [ ]:
dem_data = f'{Path.home()}/gda_demo_data/dem_data'
dem_fn = os.path.join(dem_data, "WA_COP90.tif")

In [ ]:
proj_fn = os.path.splitext(dem_fn)[0]+'_utm_gdalwarp.tif'
slope_fn = os.path.splitext(proj_fn)[0]+'_slope.tif'

In [ ]:
!gdaldem slope $proj_fn $slope_fn

In [ ]:
slope_ras = rxr.open_rasterio(slope_fn, masked=True).squeeze()

### Substation Distance

In [ ]:
#This Code was aquired using the help of ChatGPT
# URL of the ArcGIS FeatureServer
url = "https://services6.arcgis.com/OO2s4OoyCZkYJ6oE/arcgis/rest/services/Substations/FeatureServer/0/query"

resolution = 100 

# Define query parameters for the request
params = {
    "where": "COUNTY = 'SPOKANE'",  # Filter for records where the STATE field is 'WA'
    "outFields": "*",  # Fetch all fields
    "f": "geojson"  # Request GeoJSON format
}

# Send the request and get the response
response = requests.get(url, params=params)

# Check if the request was successful
if response.status_code == 200:
    # Load the GeoJSON into a GeoDataFrame
    substation_gdf = gpd.read_file(response.text)

else:
    print(f"Failed to fetch data: {response.status_code}")

In [ ]:
# Code aquired from CHAT
substations = substation_gdf

substations = substations.to_crs(epsg=32610)
substations = substations.to_crs(proj_str_aea)
tract_geom_gdf =tract_geom_gdf.to_crs(proj_str_aea)

# Get the bounding box from substations
xmin, ymin, xmax, ymax = tract_geom_gdf.total_bounds

# Define the resolution of the raster (in meters)
resolution = 100  # Change this as needed (e.g., 100 meters per pixel)
width = int((xmax - xmin) / resolution)
height = int((ymax - ymin) / resolution)


raster = np.full((height, width), np.nan, dtype=np.float32)

# Extract coordinates of substations
substation_coords = np.array(list(zip(substations.geometry.x, substations.geometry.y)))

# Create a spatial index using cKDTree for efficient distance computation
tree = cKDTree(substation_coords)

# Compute distances for each pixel
for i in range(height):
    for j in range(width):
        x = xmin + j * resolution + resolution / 2
        y = ymax - i * resolution - resolution / 2
        dist, _ = tree.query((x, y))
        raster[i, j] = dist / 1000  # converts it to km


transform = from_origin(xmin, ymax, resolution, resolution)

# Save raster to file
with rio.open(
    "./Data/distance_raster_proj_alt.tif", "w", driver="GTiff", height=height, width=width,
    count=1, dtype=np.float32, crs=proj_str_aea, transform=transform
) as dst:
    dst.write(raster, 1)

In [ ]:
substation_distance_raster_fn = "./Data/distance_raster_proj_alt.tif"
substation_distance_raster_data = rxr.open_rasterio(substation_distance_raster_fn).squeeze()

### Radiation

In [ ]:
## Only run once!

radiation_fn = "./Data/radiation.tif"
radiation = rxr.open_rasterio(radiation_fn).squeeze()

### Land Use

In [ ]:
land_use_fn = "./Data/land_use.tif"
proj_land_fn = os.path.splitext(land_use_fn)[0]+'_utm_gdalwarp.tif'

In [ ]:
dst_crs = 'EPSG:4326'

!gdalwarp -srcnodata 0 -t_srs $dst_crs $land_use_fn $proj_land_fn

In [ ]:
land_use_da = rxr.open_rasterio(proj_land_fn).squeeze()

---
## Section 2A: Data Prep

### CRS Projection Match to Spokane AEA

In [ ]:
slope_ras = slope_ras.rio.reproject(proj_str_aea)

land_use_da = land_use_da.rio.reproject(proj_str_aea)

radiation = radiation.rio.reproject(proj_str_aea)

census_Spokane_gdf = census_Spokane_gdf.to_crs(proj_str_aea)

tract_geom_gdf = tract_geom_gdf.to_crs(proj_str_aea)

floodplains_gdf = floodplains_gdf.to_crs(proj_str_aea)

substation_gdf = substation_gdf.to_crs(proj_str_aea)

census_geom = census_Spokane_gdf.geometry.unary_union

### Clipping all data to available land in Spokane

In [ ]:
slope = slope_ras.rio.clip([census_geom])
land_use = land_use_da.rio.clip([census_geom])
radiation = radiation.rio.clip([census_geom])
substation_distance = substation_distance_raster_data.rio.clip([census_geom])
floodplain_buffer = floodplain_buffer_data.rio.clip([census_geom])

### Matching Resolutions

In [ ]:
land_use = land_use_da.rio.reproject_match(floodplain_buffer_data)
radiation = radiation.rio.reproject_match(floodplain_buffer_data)
slope = slope.rio.reproject_match(floodplain_buffer_data)

In [ ]:
print(radiation.rio.resolution(), 
      slope.rio.resolution(), 
      land_use.rio.resolution(),
      substation_distance.rio.resolution(),
      floodplain_buffer.rio.resolution())

# Plotting

In [ ]:
ds = [radiation, slope, land_use, substation_distance, floodplain_buffer]
titles = ['Radiation', 'Slope', 'Land Use', 'Substation Distance', 'Floodplain Buffer']
cmap = ['magma', 'terrain_r', 'terrain_r', 'viridis', 'magma']

n_rows = (len(ds) + 1) // 2

fig, axs = plt.subplots(n_rows, 2, figsize=(12, n_rows * 5))

# Flatten axs in case there's only one row
axs = axs.flatten()

for i, (data, title, cmap) in enumerate(zip(ds, titles, cmap)):
    data.plot(ax=axs[i], cmap=cmap)
    axs[i].set_title(title)
    axs[i].axis('on')

for j in range(i + 1, len(axs)):
    fig.delaxes(axs[j])

plt.tight_layout()
plt.show()

---
## Section 2B: Merging all data to XDarray Datafrane & Summary Table

In [ ]:
floodplain_buffer.name = 'Flood_Zones'
substation_distance.name = 'Substation_Distance'
radiation.name = 'Radiation'
land_use.name = 'Land_Use'
slope.name = 'Slope'

In [ ]:
merged_df = xr.merge([floodplain_buffer, 
                      slope, 
                      substation_distance, 
                      radiation, 
                      land_use])
merged_df

### Summary Table

In [ ]:
st = merged_df.mean()
st = merged_df.to_dataframe().describe()
st

## Section 3: Classifying & Scaling DF 

In [ ]:
scale = merged_df

In [ ]:
scale['Land_Use'] = xr.where((scale['Land_Use'] >= 150) & (scale['Land_Use'] <= 180) | (scale['Land_Use'] >= 210), 1, scale['Land_Use'])  # Water/Snow
scale['Land_Use'] = xr.where((scale['Land_Use'] == 190), 2, scale['Land_Use'])  # Urban Areas
scale['Land_Use'] = xr.where((scale['Land_Use'] >= 10) & (scale['Land_Use'] <= 20), 3, scale['Land_Use'])  # Farmland
scale['Land_Use'] = xr.where((scale['Land_Use'] >= 50) & (scale['Land_Use'] <= 90) | (scale['Land_Use'] == 100), 4, scale['Land_Use'])  # Forests
scale['Land_Use'] = xr.where((scale['Land_Use'] == 200), 5, scale['Land_Use'])  # Bare Areas
scale['Land_Use'] = xr.where((scale['Land_Use'] == 30) | (scale['Land_Use'] >= 110) & (scale['Land_Use'] <= 140), 6, scale['Land_Use']) # Grasslands

In [ ]:
# Creates the scale for the substation distancce. Based on how far the substation is from each pixel
scale["Flood_Zones"] = merged_df.Flood_Zones * 6
# Always reference original values from merged_df, ensuring that things are not being overwritten in the original
original_values = merged_df["Substation_Distance"]

# Only update values in scale without overwriting previous changes
scale["Substation_Distance"] = xr.where(original_values <= 2.5, 6, scale["Substation_Distance"])
scale["Substation_Distance"] = xr.where((original_values > 2.5) & (original_values <= 5), 5, scale["Substation_Distance"])
scale["Substation_Distance"] = xr.where((original_values > 5) & (original_values <= 7.5), 4, scale["Substation_Distance"])

scale["Substation_Distance"] = xr.where((original_values >= 7.5) & (original_values < 10), 3, scale["Substation_Distance"])
scale["Substation_Distance"] = xr.where((original_values > 10) & (original_values <= 15), 2, scale["Substation_Distance"])
scale["Substation_Distance"] = xr.where((original_values > 15), 1, scale["Substation_Distance"])

In [ ]:

original_values = merged_df["Slope"]

# Slope is ranked by the best, 6, being the lowest slopes, and the worst, one being the steepest
scale["Slope"] = xr.where(original_values <= 5, 6, scale["Slope"])
scale["Slope"] = xr.where((original_values > 5) & (original_values <= 10), 5, scale["Slope"])
scale["Slope"] = xr.where((original_values > 10) & (original_values <= 15), 4, scale["Slope"])

#Any slope above 15 too steep and gets a value of one
scale["Slope"] = xr.where(original_values >= 15, 1, scale["Slope"])


In [ ]:
min_value = merged_df["Radiation"].min().item()
max_value = merged_df["Radiation"].max().item()
range_values = (max_value - min_value)/ 6

In [ ]:
original_values = merged_df["Radiation"]

#Values seperated ransomly as the values are all within range for what is desired from a solar farm

scale["Radiation"] = xr.where((original_values > (max_value-range_values)) & (original_values <= max_value), 6, scale["Radiation"])
scale["Radiation"] = xr.where((original_values > (max_value-range_values*2)) & (original_values <= (max_value - range_values)), 5, scale["Radiation"])
scale["Radiation"] = xr.where((original_values > (max_value - range_values*3)) & (original_values <= (max_value - 2*range_values)), 4, scale["Radiation"])
scale["Radiation"] = xr.where((original_values >= (min_value+ 2* range_values)) & (original_values <= (min_value + 3*range_values)), 3, scale["Radiation"])
scale["Radiation"] = xr.where((original_values >= (min_value+range_values)) & (original_values <= (min_value + 2*range_values)), 2, scale["Radiation"])
scale["Radiation"] = xr.where((original_values >= (min_value)) & (original_values <= (min_value + range_values)), 1, scale["Radiation"])

In [ ]:
## removes all the spots that are ether nan or 0 as those are the areas that are either outside the county border 
##    or have been marksed as flood plains. Removes these points from every raster.
scale = scale.where((scale.Flood_Zones.notnull()) & (scale.Flood_Zones != 0), drop=True)

In [ ]:
ds = [scale["Radiation"], scale["Slope"], scale["Land_Use"], scale["Substation_Distance"], scale["Flood_Zones"]]
titles = ['Radiation', 'Slope', 'Land Use', 'Substation Distance', 'Floodplain Buffer']
cmap = ['inferno', 'Greens', 'Greens_r', 'viridis', 'inferno']

n_rows = (len(ds) + 1) // 2

fig, axs = plt.subplots(n_rows, 2, figsize=(12, n_rows * 5))

# Flatten axs in case there's only one row
axs = axs.flatten()

for i, (data, title, cmap) in enumerate(zip(ds, titles, cmap)):
    data.plot(ax=axs[i], cmap=cmap)
    axs[i].set_title(title)
    axs[i].axis('on')

for j in range(i + 1, len(axs)):
    fig.delaxes(axs[j])

plt.tight_layout()
plt.show()

## Section 4: Score Weight Calculation

In [ ]:
# It was determined that the score would be weighted based on the importance/influence of the variable
# The weight was based on what we thoguht was most important but was also somewhat arbitrary

# score = land use * 1 + substatin distacne * 1 + radiation * 0 .75 + slope * 1.25

net_score_unweighted = scale["Radiation"] + scale["Slope"] + scale["Substation_Distance"] + scale["Flood_Zones"]
net_score_weighted = scale["Radiation"] * 0.75 + scale["Slope"] * 1.25 + scale["Substation_Distance"] * 1 + scale["Flood_Zones"] * 0.25

scale["net_score_weighted"] = net_score_weighted
scale['net_score_unweighted']= net_score_unweighted


In [ ]:
fig,ax = plt.subplots()
scale.net_score_weighted.plot(ax=ax)
tract_geom_gdf.plot(ax=ax, edgecolor="black", facecolor = 'none')

substation_gdf.plot(ax=ax, color='red', edgecolor='k',alpha=0.7)


In [ ]:
scale.net_score_unweighted.plot()

In [ ]:
print(scale_filtered)

## Section 5: Finding 200 acre-ft areas

In [ ]:
scale_filtered = scale.where((scale.net_score_weighted > 22), drop=True)
scale_filtered.net_score_weighted.plot.imshow(vmin = 0, vmax= 22)

In [ ]:
scale_filtered

In [ ]:

ds = scale_filtered

import numpy as np
import xarray as xr
import geopandas as gpd
from shapely.geometry import box, Polygon
from scipy.ndimage import label

# Load your dataset (assuming it's already loaded as 'ds')
# ds = xr.open_dataset("your_dataset.nc")  # Uncomment if loading from a file

# 404686 ft^2 is 100 acres
TARGET_AREA_M2 = 80937.1

# Get spatial resolution
res_x = np.abs(ds.x[1] - ds.x[0])  # Grid cell width
res_y = np.abs(ds.y[1] - ds.y[0])  # Grid cell height
cell_area = res_x * res_y

# Calculate target number of grid cells
target_cells = int(TARGET_AREA_M2 / cell_area)

# Get net score array
net_score = ds.net_score_weighted.values

# Create a binary mask for valid data (exclude NaNs)
binary_mask = np.where(~np.isnan(net_score), 1, 0)

# Label contiguous regions
labeled_array, num_features = label(binary_mask)

# List to store results
results = []

# Iterate through labeled regions
for region_id in range(1, num_features + 1):
    region_mask = labeled_array == region_id
    
    # Check total area
    if np.sum(region_mask) < target_cells:
        continue
    
    # Identify bounding box
    y_indices, x_indices = np.where(region_mask)
    x_min_idx, x_max_idx = np.min(x_indices), np.max(x_indices)
    y_min_idx, y_max_idx = np.min(y_indices), np.max(y_indices)
    
    # Ensure at least 150 pixels wide
    if (x_max_idx - x_min_idx) < 10:
        continue
    
    # Get coordinates
    x_min, x_max = ds.x[x_min_idx].values, ds.x[x_max_idx].values
    y_min, y_max = ds.y[y_max_idx].values, ds.y[y_min_idx].values
    
    # Create geometry
    geom = box(x_min, y_min, x_max, y_max)
    
    # Compute total score
    score = np.nansum(net_score[region_mask])
    
    # Store result
    results.append({"score": score, "geometry": geom})

# Convert results to a GeoDataFrame
gdf = gpd.GeoDataFrame(results, columns=["score", "geometry"], crs=proj_str_aea)  # Adjust CRS as needed


# Print summary
print(gdf.head())

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
tract_geom_gdf.plot(ax=ax, edgecolor="black", facecolor = 'none')
gdf.plot(column="score", cmap="viridis", edgecolor="black", legend=True, ax=ax)
ax.set_title("Optimal Land Parcels")

In [ ]:
scale_filtered = scale_filtered.rio.reproject(proj_str_aea)

In [ ]:
# Import necessary libraries
import numpy as np
import xarray as xr
import geopandas as gpd
from shapely.geometry import box
from scipy.ndimage import label

# Define constants
TARGET_AREA_M2 = 80937.1  # Example: area for 100 acres in m²

# Calculate resolution
res_x = np.abs(ds.x[1] - ds.x[0])
res_y = np.abs(ds.y[1] - ds.y[0])
cell_area = res_x * res_y
target_cells = int(TARGET_AREA_M2 / cell_area)

# Create a binary mask
net_score = ds.net_score_weighted.values
binary_mask = np.where(~np.isnan(net_score), 1, 0)

# Label regions
labeled_array, num_features = label(binary_mask)

# Store results
results = []

# Iterate through labeled regions
for region_id in range(1, num_features + 1):
    region_mask = labeled_array == region_id
    
    if np.sum(region_mask) < target_cells:
        continue
    
    y_indices, x_indices = np.where(region_mask)
    x_min_idx, x_max_idx = np.min(x_indices), np.max(x_indices)
    y_min_idx, y_max_idx = np.min(y_indices), np.max(y_indices)
    
    if (x_max_idx - x_min_idx) < 10:
        continue

    # Convert to spatial coordinates
    x_min, x_max = ds.x[x_min_idx].values, ds.x[x_max_idx].values
    y_min, y_max = ds.y[y_max_idx].values, ds.y[y_min_idx].values
    
    geom = box(x_min, y_min, x_max, y_max)
    score = np.nansum(net_score[region_mask])
    
    results.append({"score": score, "geometry": geom})

# Create GeoDataFrame
gdf = gpd.GeoDataFrame(results, columns=["score", "geometry"], crs=4326)

# Print summary
print(gdf.head())

In [ ]:
f, ax = plt.subplots(figsize=(9,6))
gdf.plot(column="score", cmap="viridis", edgecolor="black", legend=True, ax=ax)
ax.set_aspect('equal')  # Fix aspect ratio to be equal
plt.show()

In [ ]:
ds = scale_filtered

import numpy as np
import xarray as xr
import geopandas as gpd
from shapely.geometry import box

# Load your dataset (assuming it's already loaded as 'ds')
# ds = xr.open_dataset("your_dataset.nc")  # Uncomment if loading from a file

# Convert 200 acres to square meters
TARGET_AREA_M2 = 809372

# Get spatial resolution
res_x = np.abs(ds.x[1] - ds.x[0])  # Grid cell width
res_y = np.abs(ds.y[1] - ds.y[0])  # Grid cell height
cell_area = res_x * res_y

# Calculate target number of grid cells
target_cells = int(TARGET_AREA_M2 / cell_area)

# Possible aspect ratios (width, height)
aspect_ratios = [(1, 1), (2, 1), (1, 2), (4, 1), (1, 4)]  # Can be adjusted

# Get net score array
net_score = ds.net_score_weighted.values

# List to store results
results = []

# Iterate over aspect ratios
for w_ratio, h_ratio in aspect_ratios:
    width_cells = int(np.sqrt(target_cells * w_ratio / h_ratio))
    height_cells = int(target_cells / width_cells)
    
    # Ensure valid size
    if width_cells <= 0 or height_cells <= 0:
        continue
    
    # Sliding window search
    for i in range(net_score.shape[0] - height_cells + 1):
        for j in range(net_score.shape[1] - width_cells + 1):
            
            # Extract rectangle and compute score
            score = np.nansum(net_score[i:i+height_cells, j:j+width_cells])
            
            # Ensure indices are within bounds
            x_max_idx = min(j + width_cells, len(ds.x) - 1)
            y_max_idx = min(i + height_cells, len(ds.y) - 1)
            
            # Get coordinates safely
            x_min, x_max = ds.x[j].values, ds.x[x_max_idx].values
            y_min, y_max = ds.y[y_max_idx].values, ds.y[i].values
            
            # Create geometry
            geom = box(x_min, y_min, x_max, y_max)
            
            # Store result
            results.append({"score": score, "geometry": geom})

# Convert results to a GeoDataFrame
gdf = gpd.GeoDataFrame(results, columns=["score", "geometry"], crs="EPSG:4326")  # Adjust CRS as needed

# Print summary
print(gdf.head())

# Random Plots

In [ ]:
# colors = {
#     1: 'lightblue',   # water/snow
#     2: 'sienna',      # urban
#     3: 'red', # farm
#     4: 'forestgreen', # forests
#     5: 'red',         # bare
#     6: 'red'          # grasslands
# }

# cmap = mcolors.ListedColormap([colors[i] for i in range(1, 7)])

# norm = mcolors.BoundaryNorm(np.arange(0.5, 7.5, 1), cmap.N)

# fig, ax = plt.subplots(figsize=(9, 6))
# im = land_classes.plot.imshow(ax=ax, cmap=cmap, norm=norm, add_colorbar=False)

# legend_patches = [
#     Patch(color=colors[1], label='Water & Snow'),
#     Patch(color=colors[2], label='Urban Areas'),
#     Patch(color=colors[3], label='Farm Land'),
#     Patch(color=colors[4], label='Forest Cover'),
#     Patch(color=colors[5], label='Bare Areas'),
#     Patch(color=colors[6], label='Grasslands')
# ]

# tract_geom_gdf.plot(ax=ax, edgecolor="black", facecolor = 'none')
# ax.legend(handles=legend_patches, bbox_to_anchor=(1, 1), loc='upper left', title='Land Use Classes')
# ax.set_title('Land Cover')

# plt.show()

In [ ]:
# fig,ax = plt.subplots()

# radiation.plot.imshow(ax=ax)
# tract_geom_gdf.plot(ax=ax, edgecolor="black", facecolor = 'none')

# plt.title("Spokane UV Map")
# plt.show()

In [ ]:
# fig,ax = plt.subplots(figsize=(9,7))

# substation_distance_raster_data.plot(ax=ax)
# tract_geom_gdf.plot(ax=ax, edgecolor="black", facecolor = 'none')

# substation_gdf.plot(ax=ax, color='red', edgecolor='k',alpha=0.7)

# plt.suptitle('Substation distance')

In [ ]:
# fig,ax = plt.subplots()
# tract_geom_gdf.plot(ax=ax, edgecolor="black", facecolor = 'none')
# census_Spokane_gdf.plot(ax=ax, column="TotalPopulation")
# plt.title("Population Density")
# plt.show()

In [ ]:
# fig,ax = plt.subplots()

# slope_Spokane.plot.imshow(ax=ax, alpha=0.75, vmin=0, vmax = 40,cmap="magma_r")
# tract_geom_gdf.plot(ax=ax, edgecolor="black", facecolor = 'none')
# plt.title("Spokane County Slope Map")
# plt.show()

In [ ]:
# f,ax=plt.subplots()
# floodplains_gdf.plot(ax=ax)
# tract_geom_gdf.plot(ax=ax, edgecolor="black", facecolor = 'none')

In [ ]:
# fig,ax = plt.subplots()

# #floodplains_gdf.plot(ax=ax)
# tract_geom_gdf.plot(ax=ax, edgecolor="black", facecolor = 'none')
# slope_Spokane.plot.imshow(ax=ax, alpha=0.5, vmin=0, vmax = 40,cmap="magma_r")

# plt.title("Spokane County: Flood Plain Overlapped with Flood Plains")
# plt.show()

In [ ]:
# fig,ax = plt.subplots()

# tract_geom_gdf.plot(ax=ax, edgecolor="black", facecolor = 'none')
# substation_gdf.plot(ax=ax, color='blue', edgecolor='black')

# plt.title('Substations in Washington (WA)')

In [ ]:
# fig,ax = plt.subplots()

# substation_gdf.plot(ax=ax, color='blue', edgecolor='black')
# radiation.plot.imshow(ax = ax, alpha=0.75)
# floodplains_gdf.plot(ax=ax)
# tract_geom_gdf.plot(ax=ax, edgecolor="black", facecolor = 'none')
# slope_Spokane.plot.imshow(ax=ax, alpha=0.5, vmin=0, vmax = 40,cmap="magma_r")

In [ ]:
# fig,ax = plt.subplots()

# floodplain_binary_data.plot(ax = ax)
# tract_geom_gdf.plot(ax = ax, edgecolor="black", facecolor = 'none')